In [ ]:
import os
from pathlib import Path


# Navigate to the project root directory (where .git is)
def find_git_root(path="."):
    path = Path(path).resolve()
    for parent in [path] + list(path.parents):
        if (parent / ".git").exists():
            return parent
    return path  # fallback if .git not found


project_root = find_git_root()
os.chdir(project_root)
print(f"Working directory set to: {project_root}")

Working directory set to: /Users/sherman/GitHub/ISYE-6740/Final Project/Workspace


In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

pd.set_option("display.max_columns", 50)

In [ ]:
master_data = pd.read_parquet("data/processed/master_dataset.parquet")
master_data.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,ProductName,Price,CategoryID,Class,ModifyDate,Resistant,IsAllergic,VitalityDays,CategoryName,SalesPersonFirstName,SalesPersonMiddleInitial,SalesPersonLastName,SalesPersonBirthDate,SalesPersonGender,SalesPersonCityID,SalesPersonHireDate,SalesPersonCityName,SalesPersonZipcode,SalesPersonCountryID,SalesPersonCountryName,SalesPersonCountryCode,CustomerFirstName,CustomerMiddleInitial,CustomerLastName,CustomerCityID,CustomerAddress,CustomerCityName,CustomerZipcode,CustomerCountryID,CustomerCountryName,CustomerCountryCode
0,1,6,27039,381,7,0.0,0.0,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,Vaccum Bag 10x13,44.2337,1,High,2018-01-06 22:26:53.580,Unknown,Unknown,41.0,Confections,Holly,E,Collins,1987-01-13 00:00:00.000,M,65,2013-06-22 13:20:18.080,Baltimore,89197,32,United States,AR,Susan,V,Green,54,826 Rocky Second Freeway,Albuquerque,55358,32,United States,AR
1,2,16,25011,61,7,0.0,0.0,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,Sardines,62.5460,8,Low,2017-12-03 21:41:12.420,Durable,False,90.0,Grain,Chadwick,U,Walton,1951-07-07 00:00:00.000,M,28,2017-02-10 11:21:26.650,Tucson,34760,32,United States,AR,Telly,W,Pollard,71,334 Cowley Street,Phoenix,84193,32,United States,AR
2,3,13,94024,23,24,0.0,0.0,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,Crab - Imitation Flakes,79.0184,11,Medium,2017-08-19 07:18:29.890,Durable,True,0.0,Produce,Katina,Y,Marks,1963-04-18 00:00:00.000,M,68,2011-12-12 10:43:52.940,Anchorage,29493,32,United States,AR,Jon,K,Rangel,2,537 First St.,Buffalo,17420,32,United States,AR
3,4,8,73966,176,19,0.2,0.0,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,Smirnoff Green Apple Twist,81.3167,6,High,2017-08-27 17:08:59.840,Durable,True,90.0,Seafood,Julie,E,Dyer,1956-12-13 00:00:00.000,M,18,2014-10-14 23:12:53.420,Little Rock,81251,32,United States,AR,Carol,V,Gilmore,45,41 Hague Way,Dallas,20151,32,United States,AR
4,5,10,32653,310,9,0.0,0.0,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,Coffee - Dark Roast,79.9780,9,Low,2018-01-14 19:33:15.530,Weak,False,0.0,Poultry,Jean,P,Vang,1963-12-30 00:00:00.000,M,9,2012-07-23 15:02:12.640,Atlanta,66212,32,United States,AR,Terra,V,Carter,82,18 East Second Boulevard,Charlotte,39774,32,United States,AR


In [ ]:
# TotalPrice is all 0.0
# Price is price before discount
# Recalculate TotalPrice
master_data["TotalPrice"] = (
    master_data["Quantity"] * master_data["Price"] * (1 - master_data["Discount"])
)

In [ ]:
# All customers are from US
master_data["CustomerCountryName"].unique()

array(['United States'], dtype=object)

In [ ]:
# All sales person are from US
master_data["SalesPersonCountryName"].unique()

array(['United States'], dtype=object)

In [ ]:
master_data["SalesDate"] = pd.to_datetime(master_data["SalesDate"])
master_data["SalesDate"].min(), master_data["SalesDate"].max()

(Timestamp('2018-01-01 00:00:04.070000'),
 Timestamp('2018-05-09 23:59:59.400000'))

In [ ]:
customer_dim = master_data.filter(like="Customer", axis=1).columns.tolist()

customer_indexed = (
    master_data.sort_values(customer_dim + ["SalesDate"])
    .groupby(customer_dim, dropna=False)
    .agg(list)
    .reset_index()
)

customer_indexed.head()

,CustomerID,CustomerFirstName,CustomerMiddleInitial,CustomerLastName,CustomerCityID,CustomerAddress,CustomerCityName,CustomerZipcode,CustomerCountryID,CustomerCountryName,CustomerCountryCode,SalesID,SalesPersonID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,ProductName,Price,CategoryID,Class,ModifyDate,Resistant,IsAllergic,VitalityDays,CategoryName,SalesPersonFirstName,SalesPersonMiddleInitial,SalesPersonLastName,SalesPersonBirthDate,SalesPersonGender,SalesPersonCityID,SalesPersonHireDate,SalesPersonCityName,SalesPersonZipcode,SalesPersonCountryID,SalesPersonCountryName,SalesPersonCountryCode
0,1,Stefanie,Y,Frye,79,97 Oak Avenue,Oklahoma,40472,32,United States,AR,"[452499, 514072, 4719498, 167492, 2299013, 291...","[9, 5, 13, 12, 20, 4, 21, 15, 18, 10, 2, 16, 1...","[214, 320, 76, 125, 326, 361, 302, 37, 384, 41...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, ...","[61.2121, 73.4396, 65.6765, 6.1417, 38.859, 36...","[2018-01-03 05:24:59.690000, 2018-01-04 13:53:...","[17URA7QKGLD0BBKENLWZ, UZW8CM2KHEH3V4Q7TJ6P, 6...","[French Pastry - Mini Chocolate, Yogurt - Fren...","[61.2121, 73.4396, 65.6765, 6.1417, 38.859, 36...","[5, 9, 6, 9, 9, 2, 11, 7, 10, 5, 6, 8, 7, 7, 3...","[Low, Low, Low, Low, High, Medium, Medium, Low...","[2018-03-09 18:56:56.000, 2017-12-22 23:43:02....","[Durable, Unknown, Durable, Weak, Unknown, Wea...","[Unknown, Unknown, False, False, False, True, ...","[0.0, 0.0, 95.0, 0.0, 0.0, 71.0, 0.0, 0.0, 105...","[Beverages, Poultry, Seafood, Poultry, Poultry...","[Daphne, Desiree, Katina, Lindsay, Shelby, Dar...","[X, L, Y, M, P, O, D, D, C, P, W, U, T, O, U, ...","[King, Stuart, Marks, Chen, Riddle, Nielsen, B...","[1956-05-02 00:00:00.000, 1963-05-03 00:00:00....","[F, F, M, F, M, M, M, M, M, M, F, M, F, F, M, ...","[39, 23, 68, 58, 14, 39, 65, 92, 58, 9, 4, 28,...","[2013-04-17 14:48:02.700, 2014-11-16 22:59:54....","[Lubbock, Anaheim, Anchorage, Columbus, Indian...","[58464, 21500, 29493, 87987, 81678, 58464, 891...","[32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 3...","[United States, United States, United States, ...","[AR, AR, AR, AR, AR, AR, AR, AR, AR, AR, AR, A..."
1,2,Sandy,T,Kirby,96,52 White First Freeway,Pittsburgh,14257,32,United States,AR,"[5103377, 3067129, 4650224, 6559918, 178606, 3...","[10, 5, 11, 4, 2, 11, 16, 3, 13, 22, 11, 16, 1...","[180, 90, 136, 136, 176, 142, 234, 365, 74, 52...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.2, ...","[30.8932, 69.32, 24.3913, 24.3913, 73.18503, 2...","[2018-01-01 09:25:58.360000, 2018-01-03 16:40:...","[JISIZ26DMZGWP9XMD11J, 63D7YWNK1ECQG42PQ23X, B...","[Chocolate - Dark, Flavouring - Orange, Vol Au...","[30.8932, 69.32, 24.3913, 24.3913, 81.3167, 24...","[3, 9, 10, 10, 6, 6, 1, 7, 7, 1, 7, 8, 5, 3, 1...","[Low, High, Medium, Medium, High, Medium, Medi...","[2018-03-01 00:44:38.640, 2018-04-30 21:22:41....","[Durable, Unknown, Weak, Weak, Durable, Weak, ...","[True, True, True, True, True, True, Unknown, ...","[27.0, 37.0, 0.0, 0.0, 90.0, 0.0, 0.0, 95.0, 2...","[Cereals, Poultry, Snails, Snails, Seafood, Se...","[Jean, Desiree, Sonya, Darnell, Christine, Son...","[P, L, E, O, W, E, U, Y, Y, O, E, U, T, X, D, ...","[Vang, Stuart, Dickson, Nielsen, Palmer, Dicks...","[1963-12-30 00:00:00.000, 1963-05-03 00:00:00....","[M, F, F, M, F, F, M, M, M, F, F, M, F, F, M, ...","[9, 23, 12, 39, 4, 12, 28, 70, 68, 53, 12, 28,...","[2012-07-23 15:02:12.640, 2014-11-16 22:59:54....","[Atlanta, Anaheim, Tacoma, Lubbock, Fremont, T...","[66212, 21500, 43085, 58464, 20641, 43085, 347...","[32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 3...","[United States, United States, United States, ...","[AR, AR, AR, AR, AR, AR, AR, AR, AR, AR, AR, A..."
2,3,Lee,T,Zhang,55,921 White Fabien Avenue,Houston,95800,32,United States,AR,"[3609420, 571020, 6666380, 4509489, 1679709, 6...","[13, 9, 1, 18, 23, 18, 9, 11, 22, 1, 21, 8, 21...","[323, 205, 435, 176, 229, 1

In [16]:
customer_indexed.to_parquet("data/processed/customer_indexed.parquet", index=False)